In [2]:
import requests
from bs4 import BeautifulSoup
import csv

# Tạo một tệp CSV để lưu các liên kết
csv_filename = "links.csv"

# Mở tệp CSV để ghi dữ liệu
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["URL"])  # Ghi tiêu đề của cột

    # URL của trang bạn muốn cào
    url = "https://www.serenataflowers.com/all-flowers"

    # Lấy nội dung của trang web
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Tìm tất cả các thẻ <meta> có thuộc tính itemprop="url"
    meta_tags = soup.find_all("meta", itemprop="url")

    # Lặp qua các thẻ <meta> và lấy giá trị của thuộc tính "content"
    for meta_tag in meta_tags:
        url = meta_tag.get("content")
        csv_writer.writerow([url])

print("Dữ liệu đã được ghi vào tệp CSV: " + csv_filename)


Dữ liệu đã được ghi vào tệp CSV: links.csv


In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# Tạo một danh sách để lưu thông tin sản phẩm
products = []

# Đọc tệp CSV và lấy danh sách các liên kết
df = pd.read_csv('links.csv')
links = df['URL']

# Lặp qua từng liên kết
for link in links:
    # Tải nội dung của liên kết
    response = requests.get(link)
    if response.status_code == 200:
        page_content = response.text

        soup = BeautifulSoup(page_content, 'html.parser')

        # Tạo một từ điển để lưu thông tin sản phẩm hiện tại
        product = {}

        # name
        product['name'] = soup.find('h1', itemprop='name', class_='product-page-title').text

        description_element = soup.find('p', itemprop='description')

        # original price
        original_price_element = soup.find('span', class_='compare-price hide-for-small-only')

        if original_price_element is not None:
            product['original_price'] = original_price_element.text

        # price
        product['price'] = soup.find('span', class_='price hide-for-small-only').text

        description_element = soup.find('p', itemprop='description')

        # description
        if description_element is not None:
            product['description'] = description_element.get_text()

        # details
        ul_element = soup.find('ul', class_='tick-list')

        if ul_element is not None:
            list_items = ul_element.find_all('li')
            details = [item.get_text().strip() for item in list_items]
            product['details'] = details

        # delivery
        tab3 = soup.find('div', class_='panel', id='tab-3-panel')

        if tab3 is not None:
            product['delivery'] = tab3.get_text()

        # sub info
        tab4 = soup.find('div', class_='panel', id='tab-4-panel')

        if tab4 is not None:
            product['sub_info'] = tab4.get_text()

        # link image
        target_element = soup.find(class_='grid-x subscription-tiles product-thumbs')
        hrefs = [a['href'] for a in target_element.find_all('a') if 'href' in a.attrs][:5]  # Lấy chỉ 5 liên kết
        product['images'] = hrefs

        # Overall Rating
        rate_element = soup.find('span', class_='review-total')

        if rate_element is not None:
            product['overall_rating'] = rate_element.get_text()

        # Reviews
        reviews = []
        review_elements = soup.find_all('div', class_='cell testimonial medium-6')

        for review in review_elements:
            rate = review.get('reviewrating')

            if rate is not None:
                review_info = {}
                review_info['rate'] = rate

                content_element = review.find('meta', itemprop='reviewBody')

                if content_element is not None:
                    review_info['content'] = content_element.get('content')

                name_element = review.find('span', itemprop='author')
                name = name_element.get_text() if name_element else None
                review_info['name'] = name

                date_element = review.find('time', itemprop='dateCreated')
                date = date_element.get_text() if date_element else None
                review_info['date'] = date

                reviews.append(review_info)

        product['reviews'] = reviews

        # Thêm sản phẩm vào danh sách sản phẩm
        products.append(product)

# Lưu danh sách sản phẩm vào tệp JSON
with open('products.json', 'w') as json_file:
    json.dump(products, json_file, indent=2)


In [3]:
import requests
from bs4 import BeautifulSoup

# Danh sách các từ khóa cho các loại hoa
flower_categories = [
    "Birthday",
    "Romantic",
    "Sympathy",
    "Anniversary",
    "Get Well",
    "Baby",
    "Thank%20you",
    "Thinking%20of%20you",
    "Congratulations",
]

categories_list = {}

# Duyệt qua từng từ khóa và tạo và duyệt liên kết
for category in flower_categories:
    # Tạo liên kết dựa trên từ khóa
    url = f'https://www.serenataflowers.com/{category.lower()}-flowers'

    # Tải nội dung trang web
    response = requests.get(url)

    # Kiểm tra xem trang đã được tải thành công không
    if response.status_code == 200:
        page_content = response.text

        # Sử dụng BeautifulSoup để phân tích nội dung HTML
        soup = BeautifulSoup(page_content, 'html.parser')

        # Tạo danh sách rỗng cho từng loại hoa
        categories_list[category] = []

        # Tìm tất cả các tên sản phẩm từ trang hiện tại và thêm vào danh sách
        product_titles = soup.find_all('span', class_='container-flex product-title')
        product_names = [title.get_text().strip() for title in product_titles]

        categories_list[category] = product_names

        print(f"Loại hoa: {category}")
        print("Danh sách sản phẩm:")
        for name in product_names:
            print(name)
        print("\n")
    else:
        print(f"Lỗi khi tải trang web cho loại hoa: {category}")


AttributeError: 'NoneType' object has no attribute 'text'